In [1]:
param_market = 'DE'

from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()

output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./item.xlsx',
                       header=0,
                       dtype=str).fillna('')

dict_vehicle = {}
list_job = []
for j in range(len(input_)):
    if input_.loc[j, 'Vehicle_Page'] != '':
        dict_vehicle[input_.loc[j, 'Item_Number']] = {}
        dict_vehicle[input_.loc[j, 'Item_Number']]['vehicle_page'] = int(input_.loc[j, 'Vehicle_Page'])
        for k in range(int(input_.loc[j, 'Vehicle_Page'])):
            list_job.append((j, k))

length = len(list_job)

print('总数量：' + str(length))
print()

# = = = = = = = = = = = = = = =

import requests

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for i in range(len(list_job)):
    work.put_nowait(i)

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        i = work.get_nowait()
               
        j, k = list_job[i]
        
        # = = = = = = = = = = = = = = =
                  
        crawler_status = 'error'
        for _ in range(101):
            try:
                data = '{"scopedContext":{"catalogDetails":{"itemId":"' + input_.loc[j, 'Item_Number'] + '","categoryId":"' + input_.loc[j, 'Category_Id'] + '","marketplaceId":"' + input_.loc[j, 'Market_Id'] + '"}}}'

                ua = UA.get_User_Agent_Requests()
                ua['Content-Type'] = 'application/json'
                
                if param_market == 'US':
                    url = 'https://www.ebay.com/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset=' + str(k*20) + '&module=COMPATIBILITY_TABLE&_ul=US&_stpos=91710&orig_cvip=true'
                elif param_market == 'DE':
                    url = 'https://www.ebay.de/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset=' + str(k*20) + '&module=COMPATIBILITY_TABLE&_ul=DE&_stpos=10115&orig_cvip=true'
                # elif param_market == 'AU': 

                resp = requests.post(url,
                                     data=data,
                                     headers=ua,
                                     proxies=Proxy.get_Proxy_Requests()).text

                resp = json.loads(resp)

                # = = = = = = = = = = = = = = =

                list_header = resp['modules']['COMPATIBILITY_TABLE']['paginatedTable']['header']['cells']

                # = = = = = = = = = = = = = = =

                if param_market == 'US' or param_market == 'AU':
                    for m in range(len(list_header)):
                        if 'Make' in list_header[m]['textSpans'][0]['text']:
                            col_make = m
                        elif 'Model' in list_header[m]['textSpans'][0]['text']:
                            col_model = m
                        elif 'Year' in list_header[m]['textSpans'][0]['text']:
                            col_year = m
                        elif 'Engine' in list_header[m]['textSpans'][0]['text']:
                            col_engine = m
                elif param_market == 'DE':
                    for m in range(len(list_header)):
                        if 'Marke' in list_header[m]['textSpans'][0]['text']:
                            col_make = m
                        elif 'Modell' in list_header[m]['textSpans'][0]['text']:
                            col_model = m
                        elif 'Baujahr' in list_header[m]['textSpans'][0]['text']:
                            col_year = m
                        elif 'Motor' in list_header[m]['textSpans'][0]['text']:
                            col_engine = m

                # = = = = = = = = = = = = = = =

                list_row = resp['modules']['COMPATIBILITY_TABLE']['paginatedTable']['rows']

                # = = = = = = = = = = = = = = =

                list_vehicle = []
                if param_market == 'US' or param_market == 'AU':
                    for row in list_row:
                        make = row['cells'][col_make]['textSpans'][0]['text'].strip()
                        model = row['cells'][col_model]['textSpans'][0]['text'].strip()
                        engine = row['cells'][col_engine]['textSpans'][0]['text'].strip()
                        year = int(row['cells'][col_year]['textSpans'][0]['text'].strip())

                        list_vehicle.append({'make': make, 'model': model, 'engine': engine, 'year': year})
                elif param_market == 'DE':
                    for row in list_row:
                        make = row['cells'][col_make]['textSpans'][0]['text'].strip()
                        model = row['cells'][col_model]['textSpans'][0]['text'].strip()
                        engine = row['cells'][col_engine]['textSpans'][0]['text'].strip()
                        list_year = row['cells'][col_year]['textSpans'][0]['text'].strip().split('-')
                        for year in list_year:
                            list_vehicle.append({'make': make, 'model': model, 'engine': engine, 'year': int(year.split('/')[0])})

                dict_vehicle[input_.loc[j, 'Item_Number']][str(k)] = list_vehicle
                
                # = = = = = = = = = = = = = = =
                
                if dict_vehicle[input_.loc[j, 'Item_Number']]['vehicle_page']+1 == len(dict_vehicle[input_.loc[j, 'Item_Number']]):
                    item_list_vehicle_1 = []
                    item_dict_vehicle_1 = {}
                    for m in range(dict_vehicle[input_.loc[j, 'Item_Number']]['vehicle_page']):
                        item_list_vehicle_1 += dict_vehicle[input_.loc[j, 'Item_Number']][str(m)]

                    for vehicle in item_list_vehicle_1:
                        if vehicle['make'] not in item_dict_vehicle_1:
                            item_dict_vehicle_1[vehicle['make']] = {vehicle['model']: [vehicle['year']]}
                        else:
                            if vehicle['model'] not in item_dict_vehicle_1[vehicle['make']]:
                                item_dict_vehicle_1[vehicle['make']][vehicle['model']] = [vehicle['year']]
                            else:
                                item_dict_vehicle_1[vehicle['make']][vehicle['model']].append(vehicle['year'])
                    
                    # = = = = = = = = = = = = = = =
                    
                    item_list_vehicle_2 = []
                    item_dict_vehicle_2 = {}
                    for m in range(dict_vehicle[input_.loc[j, 'Item_Number']]['vehicle_page']):
                        item_list_vehicle_2 += dict_vehicle[input_.loc[j, 'Item_Number']][str(m)]

                    for vehicle in item_list_vehicle_2:
                        if vehicle['make'] not in item_dict_vehicle_2:
                            item_dict_vehicle_2[vehicle['make']] = {vehicle['model']: {vehicle['engine']: [vehicle['year']]}}
                        else:
                            if vehicle['model'] not in item_dict_vehicle_2[vehicle['make']]:
                                item_dict_vehicle_2[vehicle['make']][vehicle['model']] = {vehicle['engine']: [vehicle['year']]}
                            else:
                                if vehicle['engine'] not in item_dict_vehicle_2[vehicle['make']][vehicle['model']]:
                                    item_dict_vehicle_2[vehicle['make']][vehicle['model']][vehicle['engine']] = [vehicle['year']]
                                else:
                                    item_dict_vehicle_2[vehicle['make']][vehicle['model']][vehicle['engine']].append(vehicle['year'])

                    # = = = = = = = = = = = = = = =
                    
                    item_list_vehicle_1 = []
                    for make in item_dict_vehicle_1:
                        for model in item_dict_vehicle_1[make]:
                            year_max = max(item_dict_vehicle_1[make][model])
                            year_min = min(item_dict_vehicle_1[make][model])
                            if year_max == year_min:
                                item_list_vehicle_1.append(make + ' ' + model + ' ' + str(year_min))
                            else:
                                item_list_vehicle_1.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))

                    vehicle_1 = '\n'.join(item_list_vehicle_1)
                    
                    # = = = = = = = = = = = = = = =
                
                    item_list_vehicle_2 = []
                    for make in item_dict_vehicle_2:
                        for model in item_dict_vehicle_2[make]:
                            for engine in item_dict_vehicle_2[make][model]:
                                year_max = max(item_dict_vehicle_2[make][model][engine])
                                year_min = min(item_dict_vehicle_2[make][model][engine])
                                if year_max == year_min:
                                    item_list_vehicle_2.append(make + ' ' + model + ' ' + str(year_min) + ' - ' + engine)
                                else:
                                    item_list_vehicle_2.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max) + ' - ' + engine)

                    vehicle_2 = '\n'.join(item_list_vehicle_2)

                    # = = = = = = = = = = = = = = =

                    df_temp = pd.DataFrame([{'Item_Number': input_.loc[j, 'Item_Number'],
                                             'Vehicle_1': vehicle_1,
                                             'Vehicle_2': vehicle_2}])

                    output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                    
                    print('Note：' + input_.loc[j, 'Item_Number'] + ' 已全部获取')
                    
                # = = = = = = = = = = = = = = =
               
                crawler_status = 'ok'
                
                break
                
            except:
                time.sleep(0.3)
                continue
                
        if crawler_status != 'ok':
            df_temp = pd.DataFrame([{'Item_Number': input_.loc[j, 'Item_Number'],
                                     'Vehicle_Page': input_.loc[j, 'Vehicle_Page'],
                                     'Category_Id': input_.loc[j, 'Category_Id'],
                                     'Market_Id': input_.loc[j, 'Market_Id']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        print(input_.loc[j, 'Item_Number'] + ' ' + str(k) +  '  <->  [' + crawler_status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =         
               
list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('爬虫输出中')
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Item_Number'], ascending=[True])
output_correct.to_excel('./vehicle-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Item_Number'], ascending=[True])
    output_error.to_excel('./vehicle_error.xlsx', index=False)
    print()
    print('爬虫存在error')
print()
print('爬虫结束')

总数量：7518

Note：303895332979 已全部获取
303895332979 0  <->  [ok] - 剩余数量：7498 - 当前时间：10:10:22
303895333172 1  <->  [ok] - 剩余数量：7497 - 当前时间：10:10:22
Note：303895333150 已全部获取
303895333150 0  <->  [ok] - 剩余数量：7496 - 当前时间：10:10:22
Note：303895332504 已全部获取
303895332504 0  <->  [ok] - 剩余数量：7495 - 当前时间：10:10:23
303895333109 3  <->  [ok] - 剩余数量：7494 - 当前时间：10:10:23
303895333172 3  <->  [ok] - 剩余数量：7493 - 当前时间：10:10:23
303895333120 0  <->  [ok] - 剩余数量：7492 - 当前时间：10:10:23
Note：303895330592 已全部获取
303895330592 0  <->  [ok] - 剩余数量：7491 - 当前时间：10:10:23
303895333178 1  <->  [ok] - 剩余数量：7490 - 当前时间：10:10:23
303895333109 2  <->  [ok] - 剩余数量：7489 - 当前时间：10:10:23
303895333172 4  <->  [ok] - 剩余数量：7488 - 当前时间：10:10:24
303895335713 1  <->  [ok] - 剩余数量：7487 - 当前时间：10:10:24
Note：303895335713 已全部获取
303895335713 0  <->  [ok] - 剩余数量：7486 - 当前时间：10:10:25
303895335714 2  <->  [ok] - 剩余数量：7485 - 当前时间：10:10:25
303895333172 0  <->  [ok] - 剩余数量：7484 - 当前时间：10:10:25
Note：303895333178 已全部获取
303895333178 0  <->  [ok] - 剩余数量：748

Note：303895336650 已全部获取
303895336650 2  <->  [ok] - 剩余数量：7360 - 当前时间：10:11:01
303895336683 2  <->  [ok] - 剩余数量：7359 - 当前时间：10:11:02
303895336756 2  <->  [ok] - 剩余数量：7358 - 当前时间：10:11:02
303895336635 5  <->  [ok] - 剩余数量：7357 - 当前时间：10:11:02
303895336761 0  <->  [ok] - 剩余数量：7356 - 当前时间：10:11:02
303895336664 5  <->  [ok] - 剩余数量：7355 - 当前时间：10:11:02
Note：303895336672 已全部获取
303895336672 0  <->  [ok] - 剩余数量：7354 - 当前时间：10:11:02
303895336761 4  <->  [ok] - 剩余数量：7353 - 当前时间：10:11:03
303895336761 5  <->  [ok] - 剩余数量：7352 - 当前时间：10:11:03
303895336756 3  <->  [ok] - 剩余数量：7351 - 当前时间：10:11:04
303901319739 1  <->  [ok] - 剩余数量：7350 - 当前时间：10:11:04
303895336756 0  <->  [ok] - 剩余数量：7349 - 当前时间：10:11:04
303895336664 2  <->  [ok] - 剩余数量：7348 - 当前时间：10:11:04
Note：303895336664 已全部获取
303895336664 3  <->  [ok] - 剩余数量：7347 - 当前时间：10:11:04
303921707461 0  <->  [ok] - 剩余数量：7346 - 当前时间：10:11:05
Note：303921704807 已全部获取
303921704807 0  <->  [ok] - 剩余数量：7345 - 当前时间：10:11:05
303921707461 2  <->  [ok] - 剩余数量：7344 - 

303967654430 4  <->  [ok] - 剩余数量：7226 - 当前时间：10:11:41
303967654476 4  <->  [ok] - 剩余数量：7225 - 当前时间：10:11:42
303967654430 5  <->  [ok] - 剩余数量：7224 - 当前时间：10:11:42
303967654476 3  <->  [ok] - 剩余数量：7223 - 当前时间：10:11:43
303967654476 8  <->  [ok] - 剩余数量：7222 - 当前时间：10:11:43
303967654476 5  <->  [ok] - 剩余数量：7221 - 当前时间：10:11:44
303967654476 6  <->  [ok] - 剩余数量：7220 - 当前时间：10:11:44
303967654484 3  <->  [ok] - 剩余数量：7219 - 当前时间：10:11:45
303967654438 2  <->  [ok] - 剩余数量：7218 - 当前时间：10:11:45
303967654484 0  <->  [ok] - 剩余数量：7217 - 当前时间：10:11:46
303967654484 1  <->  [ok] - 剩余数量：7216 - 当前时间：10:11:46
303967654486 1  <->  [ok] - 剩余数量：7215 - 当前时间：10:11:46
Note：303967654407 已全部获取
303967654407 0  <->  [ok] - 剩余数量：7214 - 当前时间：10:11:48
303967654486 3  <->  [ok] - 剩余数量：7213 - 当前时间：10:11:48
Note：303967654430 已全部获取
303967654430 0  <->  [ok] - 剩余数量：7212 - 当前时间：10:11:49
303967654438 1  <->  [ok] - 剩余数量：7211 - 当前时间：10:11:49
Note：303966881638 已全部获取
303966881638 1  <->  [ok] - 剩余数量：7210 - 当前时间：10:11:49
3039676544

303971488064 2  <->  [ok] - 剩余数量：7085 - 当前时间：10:12:35
Note：303971488064 已全部获取
303971488064 1  <->  [ok] - 剩余数量：7084 - 当前时间：10:12:35
303971493852 1  <->  [ok] - 剩余数量：7083 - 当前时间：10:12:36
Note：303971487932 已全部获取
303971487932 1  <->  [ok] - 剩余数量：7082 - 当前时间：10:12:36
303971493852 3  <->  [ok] - 剩余数量：7081 - 当前时间：10:12:36
Note：303971473965 已全部获取
303971473965 0  <->  [ok] - 剩余数量：7080 - 当前时间：10:12:36
Note：303971487930 已全部获取
303971487930 0  <->  [ok] - 剩余数量：7079 - 当前时间：10:12:37
303971493852 4  <->  [ok] - 剩余数量：7078 - 当前时间：10:12:37
303971494209 0  <->  [ok] - 剩余数量：7077 - 当前时间：10:12:37
303971493830 1  <->  [ok] - 剩余数量：7076 - 当前时间：10:12:38
303971494209 3  <->  [ok] - 剩余数量：7075 - 当前时间：10:12:38
303971494209 2  <->  [ok] - 剩余数量：7074 - 当前时间：10:12:38
Note：303971493830 已全部获取
303971493830 0  <->  [ok] - 剩余数量：7073 - 当前时间：10:12:38
303971494222 0  <->  [ok] - 剩余数量：7072 - 当前时间：10:12:39
303971499989 0  <->  [ok] - 剩余数量：7071 - 当前时间：10:12:40
303971499989 2  <->  [ok] - 剩余数量：7070 - 当前时间：10:12:40
Note：30397149422

304016203796 2  <->  [ok] - 剩余数量：6950 - 当前时间：10:13:16
304016203801 4  <->  [ok] - 剩余数量：6949 - 当前时间：10:13:16
Note：304016203717 已全部获取
304016203717 0  <->  [ok] - 剩余数量：6948 - 当前时间：10:13:17
304016203829 1  <->  [ok] - 剩余数量：6947 - 当前时间：10:13:17
304016203829 2  <->  [ok] - 剩余数量：6946 - 当前时间：10:13:17
Note：304016203835 已全部获取
304016203835 0  <->  [ok] - 剩余数量：6945 - 当前时间：10:13:18
Note：304016203796 已全部获取
304016203796 1  <->  [ok] - 剩余数量：6944 - 当前时间：10:13:18
304016203829 4  <->  [ok] - 剩余数量：6943 - 当前时间：10:13:19
304016203844 1  <->  [ok] - 剩余数量：6942 - 当前时间：10:13:19
304016203829 3  <->  [ok] - 剩余数量：6941 - 当前时间：10:13:19
304016203844 0  <->  [ok] - 剩余数量：6940 - 当前时间：10:13:19
304016203844 2  <->  [ok] - 剩余数量：6939 - 当前时间：10:13:20
304016203844 3  <->  [ok] - 剩余数量：6938 - 当前时间：10:13:20
304016203847 0  <->  [ok] - 剩余数量：6937 - 当前时间：10:13:20
Note：304016203844 已全部获取
304016203844 4  <->  [ok] - 剩余数量：6936 - 当前时间：10:13:21
304016203847 1  <->  [ok] - 剩余数量：6935 - 当前时间：10:13:21
304016203847 3  <->  [ok] - 剩余数量：6934 - 

...

394275352918 3  <->  [ok] - 剩余数量：610 - 当前时间：10:37:32
394251730020 7  <->  [ok] - 剩余数量：609 - 当前时间：10:37:32
Note：394288795553 已全部获取
394288795553 0  <->  [ok] - 剩余数量：608 - 当前时间：10:37:33
394275352918 5  <->  [ok] - 剩余数量：607 - 当前时间：10:37:33
394275352918 0  <->  [ok] - 剩余数量：606 - 当前时间：10:37:33
394275428393 0  <->  [ok] - 剩余数量：605 - 当前时间：10:37:34
394251730020 6  <->  [ok] - 剩余数量：604 - 当前时间：10:37:34
394289204176 3  <->  [ok] - 剩余数量：603 - 当前时间：10:37:34
394289204176 0  <->  [ok] - 剩余数量：602 - 当前时间：10:37:34
394275352918 1  <->  [ok] - 剩余数量：601 - 当前时间：10:37:35
Note：394275352918 已全部获取
394275352918 4  <->  [ok] - 剩余数量：600 - 当前时间：10:37:35
Note：394275428393 已全部获取
394275428393 1  <->  [ok] - 剩余数量：599 - 当前时间：10:37:35
Note：394269043767 已全部获取
394269043767 0  <->  [ok] - 剩余数量：598 - 当前时间：10:37:35
Note：394267694048 已全部获取
394267694048 0  <->  [ok] - 剩余数量：597 - 当前时间：10:37:35
394251730020 1  <->  [ok] - 剩余数量：596 - 当前时间：10:37:36
394289204176 1  <->  [ok] - 剩余数量：595 - 当前时间：10:37:36
394301086406 3  <->  [ok] - 剩余数量

394646473270 0  <->  [ok] - 剩余数量：471 - 当前时间：10:38:07
394646473270 1  <->  [ok] - 剩余数量：470 - 当前时间：10:38:07
394626692483 15  <->  [ok] - 剩余数量：469 - 当前时间：10:38:08
Note：394310117026 已全部获取
394310117026 2  <->  [ok] - 剩余数量：468 - 当前时间：10:38:09
Note：394646369486 已全部获取
394646369486 1  <->  [ok] - 剩余数量：467 - 当前时间：10:38:09
394646473270 4  <->  [ok] - 剩余数量：466 - 当前时间：10:38:09
394646473270 3  <->  [ok] - 剩余数量：465 - 当前时间：10:38:09
394646473270 6  <->  [ok] - 剩余数量：464 - 当前时间：10:38:09
Note：394645925181 已全部获取
394645925181 0  <->  [ok] - 剩余数量：463 - 当前时间：10:38:10
394626692483 8  <->  [ok] - 剩余数量：462 - 当前时间：10:38:10
394626692483 9  <->  [ok] - 剩余数量：461 - 当前时间：10:38:10
394646473270 8  <->  [ok] - 剩余数量：460 - 当前时间：10:38:10
394617402960 2  <->  [ok] - 剩余数量：459 - 当前时间：10:38:10
394646473270 5  <->  [ok] - 剩余数量：458 - 当前时间：10:38:11
394646473270 9  <->  [ok] - 剩余数量：457 - 当前时间：10:38:11
394646473270 12  <->  [ok] - 剩余数量：456 - 当前时间：10:38:11
Note：394448317721 已全部获取
394448317721 6  <->  [ok] - 剩余数量：455 - 当前时间：10:38:11
N

Note：394849145270 已全部获取
394849145270 0  <->  [ok] - 剩余数量：331 - 当前时间：10:38:45
394851531434 0  <->  [ok] - 剩余数量：330 - 当前时间：10:38:45
394851531434 1  <->  [ok] - 剩余数量：329 - 当前时间：10:38:46
Note：394851531434 已全部获取
394851531434 2  <->  [ok] - 剩余数量：328 - 当前时间：10:38:46
394852111659 0  <->  [ok] - 剩余数量：327 - 当前时间：10:38:46
394851531607 1  <->  [ok] - 剩余数量：326 - 当前时间：10:38:46
394851531607 0  <->  [ok] - 剩余数量：325 - 当前时间：10:38:46
394851531395 6  <->  [ok] - 剩余数量：324 - 当前时间：10:38:47
Note：394852111659 已全部获取
394852111659 1  <->  [ok] - 剩余数量：323 - 当前时间：10:38:47
Note：394851531580 已全部获取
394851531580 0  <->  [ok] - 剩余数量：322 - 当前时间：10:38:47
394852112039 0  <->  [ok] - 剩余数量：321 - 当前时间：10:38:48
Note：394851531607 已全部获取
394851531607 2  <->  [ok] - 剩余数量：320 - 当前时间：10:38:48
Note：394851531590 已全部获取
394851531590 0  <->  [ok] - 剩余数量：319 - 当前时间：10:38:48
Note：394852650748 已全部获取
394852650748 0  <->  [ok] - 剩余数量：318 - 当前时间：10:38:48
394852650760 3  <->  [ok] - 剩余数量：317 - 当前时间：10:38:49
Note：394852112039 已全部获取
394852112039 

394854287379 13  <->  [ok] - 剩余数量：194 - 当前时间：10:39:37
394854287386 0  <->  [ok] - 剩余数量：193 - 当前时间：10:39:37
394854287386 4  <->  [ok] - 剩余数量：192 - 当前时间：10:39:37
394854287379 11  <->  [ok] - 剩余数量：191 - 当前时间：10:39:38
394854649404 0  <->  [ok] - 剩余数量：190 - 当前时间：10:39:39
394854287379 16  <->  [ok] - 剩余数量：189 - 当前时间：10:39:39
394854287386 3  <->  [ok] - 剩余数量：188 - 当前时间：10:39:39
394854287386 5  <->  [ok] - 剩余数量：187 - 当前时间：10:39:40
394854287366 1  <->  [ok] - 剩余数量：186 - 当前时间：10:39:40
Note：394854287376 已全部获取
394854287376 2  <->  [ok] - 剩余数量：185 - 当前时间：10:39:40
394854649483 0  <->  [ok] - 剩余数量：184 - 当前时间：10:39:41
394854649388 1  <->  [ok] - 剩余数量：183 - 当前时间：10:39:41
394854287379 19  <->  [ok] - 剩余数量：182 - 当前时间：10:39:42
394854287379 6  <->  [ok] - 剩余数量：181 - 当前时间：10:39:42
Note：394854649483 已全部获取
394854649483 1  <->  [ok] - 剩余数量：180 - 当前时间：10:39:42
Note：394854287379 已全部获取
394854287379 17  <->  [ok] - 剩余数量：179 - 当前时间：10:39:42
394854649388 0  <->  [ok] - 剩余数量：178 - 当前时间：10:39:43
Note：394854948940 已全部获

395107735056 10  <->  [ok] - 剩余数量：51 - 当前时间：10:40:35
395121127557 0  <->  [ok] - 剩余数量：50 - 当前时间：10:40:36
395129461239 0  <->  [ok] - 剩余数量：49 - 当前时间：10:40:36
Note：395118457605 已全部获取
395118457605 1  <->  [ok] - 剩余数量：48 - 当前时间：10:40:36
395107735056 6  <->  [ok] - 剩余数量：47 - 当前时间：10:40:36
395032317905 16  <->  [ok] - 剩余数量：46 - 当前时间：10:40:37
395105720282 9  <->  [ok] - 剩余数量：45 - 当前时间：10:40:38
395107735056 7  <->  [ok] - 剩余数量：44 - 当前时间：10:40:38
395140365490 0  <->  [ok] - 剩余数量：43 - 当前时间：10:40:39
395105720282 4  <->  [ok] - 剩余数量：42 - 当前时间：10:40:39
395135699750 0  <->  [ok] - 剩余数量：41 - 当前时间：10:40:39
Note：395105720282 已全部获取
395105720282 14  <->  [ok] - 剩余数量：40 - 当前时间：10:40:39
395121127557 2  <->  [ok] - 剩余数量：39 - 当前时间：10:40:39
395129461239 2  <->  [ok] - 剩余数量：38 - 当前时间：10:40:40
395140365490 3  <->  [ok] - 剩余数量：37 - 当前时间：10:40:40
395145813164 2  <->  [ok] - 剩余数量：36 - 当前时间：10:40:41
Note：395032317905 已全部获取
395032317905 17  <->  [ok] - 剩余数量：35 - 当前时间：10:40:41
395140365490 2  <->  [ok] - 剩余数量：34 - 当前